
# Sentiment analysis from movie reviews
Import everythin we will need


In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

We pick the first 20,000 most popular interviews from the dataset to save on time.

In [ ]:

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

Loading data...


We display the data keep in mind the number actually represent words like 1 may correspond with the word "the".

In [ ]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

0 means a negative review and 1 means a positive review

What do the labels look like?

In [ ]:
y_train[0]

1

We limit the total number of words from each review at only the first 80 words once again to save on time. 


In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

Now let's set up our neural network model! Considering how complicated a LSTM recurrent neural network is under the hood, it's really amazing how easy this is to do with Keras.

We will start with an Embedding layer - this is just a step that converts the input data into dense vectors of fixed size that's better suited for a neural network. You generally see this in conjunction with index-based text data like we have here. The 20,000 indicates the vocabulary size (remember we said we only wanted the top 20,000 words) and 128 is the output dimension of 128 units.

Next we just have to set up a LSTM layer for the RNN itself. It's that easy. We specify 128 to match the output size of the Embedding layer, and dropout terms to avoid overfitting, which RNN's are particularly prone to.

Finally we just need to boil it down to a single neuron with a sigmoid activation function to choose our binay sentiment classification of 0 or 1.

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

As this is a binary classification problem, we'll use the binary_crossentropy loss function we choose the ADAM optimizer.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

We set a batch size of 32 reviews and 15 epochs(the number of times the model is trained).

In [ ]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          verbose=2,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 - 83s - loss: 0.4605 - accuracy: 0.7798 - val_loss: 0.3795 - val_accuracy: 0.8349
Epoch 2/15
25000/25000 - 80s - loss: 0.3014 - accuracy: 0.8780 - val_loss: 0.4075 - val_accuracy: 0.8237
Epoch 3/15
25000/25000 - 81s - loss: 0.2124 - accuracy: 0.9182 - val_loss: 0.4114 - val_accuracy: 0.8238
Epoch 4/15
25000/25000 - 81s - loss: 0.1520 - accuracy: 0.9445 - val_loss: 0.4581 - val_accuracy: 0.8271
Epoch 5/15
25000/25000 - 79s - loss: 0.1097 - accuracy: 0.9603 - val_loss: 0.5656 - val_accuracy: 0.8252
Epoch 6/15
25000/25000 - 78s - loss: 0.0772 - accuracy: 0.9724 - val_loss: 0.6683 - val_accuracy: 0.8210
Epoch 7/15
25000/25000 - 79s - loss: 0.0622 - accuracy: 0.9790 - val_loss: 0.7128 - val_accuracy: 0.8209
Epoch 8/15
25000/25000 - 81s - loss: 0.0366 - accuracy: 0.9877 - val_loss: 0.9249 - val_accuracy: 0.7985
Epoch 9/15
25000/25000 - 79s - loss: 0.0291 - accuracy: 0.9912 - val_loss: 0.9049 - val_accuracy: 0.8130
Epoch

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=32,
                            verbose=2)
print('Test accuracy:', acc)

25000/1 - 14s - loss: 1.3717 - accuracy: 0.8053
Test score: 1.144708181578219
Test accuracy: 0.80528


The overall accuracy is low at 80.5% however we only looked at the first 80 words of the review had this been done with all the words the accuracy would be much higher but the total time to run the model on the test data would be longer as well. 